This scraper is going to be used to find African American Film Directors from combining information from  Wikipedia and IMDB

In [45]:
from bs4 import BeautifulSoup
from imdb import Cinemagoer
import requests
import pandas as pd
import re

## This creates the Director's list from both wikipedia pages

In [46]:
url = "https://en.wikipedia.org/wiki/Category:African-American_film_directors"
url2 = "https://en.wikipedia.org/w/index.php?title=Category:African-American_film_directors&from=P"
movie_list = set()
director_films = set()
testing_list = []
names = []

name_exclude = ['Black women filmmakers', 'Pioneers of African-American Cinema']
name_check = ['film director', 'filmmaker', 'director', 'writer', 'actor', 'musician']

def director_page(url):
    try:
        result = requests.get(url)
        result.raise_for_status()
        doc = BeautifulSoup(result.text, "html.parser")
        
    except Exception as e:
        print(e)
        
    return doc.find(class_="mw-category mw-category-columns").find_all('a')


director_links = director_page(url)
director_links2 = director_page(url2)

### Creates List of (Director Names, Wikipedia Link)

In [47]:
def Directors(names, object):
    
    for director in object:
        if any(y in director.string for y in name_exclude):
            director.string = ''
        elif any(x in director.string for x in name_check):
            director.string  = director.string.rsplit('(', 1)[0]

        if director.string or director.string in names: 
            #         LIST:    NAME                       WIKI_LINK
            names.append([str(director.string.rstrip()), director['href']])
        
    return names 


names = Directors(names, director_links)
names = Directors(names, director_links2)

In [48]:
print(len(director_links2))
print(len(director_links))

114
200


### Creates the Dataframe with Name and Wiki_Link Columns

In [49]:
df = pd.DataFrame(names, columns=['Name', 'wiki_link'])
df

,Name,wiki_link
0,Abdisalam Aato,/wiki/Abdisalam_Aato
1,Fathia Absie,/wiki/Fathia_Absie
2,Anita W. Addison,/wiki/Anita_W._Addison
3,Omowale Akintunde,/wiki/Omowale_Akintunde
4,Queen Muhammad Ali,/wiki/Queen_Muhammad_Ali
...,...,...
306,Tricia Woodgett,/wiki/Tricia_Woodgett
307,Bille Woodruff,/wiki/Bille_Woodruff
308,Fronza Woods,/wiki/Fronza_Woods
309,Tanya Wright,/wiki/Tanya_Wright


### Saves the Dataframe as a csv file

In [50]:
df.to_csv("director_names.csv")

In [51]:
#print(names)

print(len(names))



311


## This is setup for getting Filmography from Director's Wikipedia page

In [98]:
director_films = set()
test = []
mxm = []

for i in range(0, len(names)):
    #wiki_name = "Aisha Taylor"  #"Neema Barnette" # # "Martin Lawrence" "Anita W. Addison" "Martin Lawrence" df.loc[i].at["Name"] 
    wiki_url = "https://en.wikipedia.org" + str(df.loc[i].at['wiki_link'])  #/wiki/" + wiki_name.replace(' ', '_')  
    #print(wiki_url)  
    table_check = False

    try:
        result = requests.get(wiki_url)
        result.raise_for_status()
        doc = BeautifulSoup(result.text, "html.parser")
    except Exception as e:
        print(e)

    if doc.find(id="Filmography") and doc.find(class_="wikitable"):
        if doc.find(id="Filmography").parent.find_next("th").find_next("th").text.rstrip() == "Title": 
            film_list = doc.find(id="Filmography").parent.find_next("th").find_all_next("tr", limit=15)        
            table_check = True
            print((wiki_url))        
        else:
            film_list = doc.find(id="Filmography").parent.find_all_next("li", limit=6)
            print(wiki_url)
    elif doc.find(id="Films") and doc.find(class_="wikitable"):
        if doc.find(id="Films").parent.find_next("th").find_next("th").text.rstrip() == "Film": 
            film_list = doc.find(id="Films").parent.find_next("th").find_all_next("tr", limit=15)        
            table_check = True
            print((wiki_url))        
        else:
            film_list = doc.find(id="Films").parent.find_all_next("li", limit=6)
            print(wiki_url)
    elif doc.find(id="Filmography"):
        film_list = doc.find(id="Filmography").parent.find_all_next("li", limit=6)
        print(wiki_url)
    elif doc.find(id="Films_2"):
        film_list = doc.find(id="Films_2").parent.find_all_next("li", limit=6)
        print(wiki_url)
    elif (doc.find(id="Films") or doc.find(id="Film")) and doc.find(class_="mw-headline"):
        if doc.find(id="Films"):
            film_list = doc.find(id="Films").parent.find_all_next("li", limit=6)
            print(wiki_url)
        else:
            film_list = doc.find(id="Film").parent.find_all_next("li", limit=6)
            print(wiki_url)
    elif (doc.find(id="Selected_filmography") or doc.find(id="Select_Filmography")) and doc.find(class_="mw-headline"):
        if doc.find(id="Selected_filmography"):
            film_list = doc.find(id="Selected_filmography").parent.find_all_next("li", limit=6)
            print(wiki_url)
        else:
            film_list = doc.find(id="Select_Filmography").parent.find_all_next("li", limit=6)
            print(wiki_url) 
    elif doc.find(id="Filmography_as_director") and doc.find(class_="mw-headline"):
        film_list = doc.find(id="Filmography_as_director").parent.find_all_next("li", limit=6)
        print(wiki_url)
    else:
        print("Couldn't find one for ", wiki_url)
        continue

    #print(film_list)
    for film in film_list:
        #print(film)
        #Filmography Section has a Film Table: Year, Movie and table check is used since it changes the format
        if table_check == True:
            #print(film.text)
            if re.search('\d\d\d\d', film.text):
                s = film.text.rsplit('\n\n')
                #print(s, s[0])

                try:
                    t = [s[1].strip(), s[0].lstrip(), wiki_url]
                    #print(s[1].strip(), s[0].strip(), "table 1")
                except:
                    s[0] = s[0].strip()
                    s[0] = re.sub('\[\d+\]',"", s[0])
                    s = s[0].split('\n')
                    #print(s)
                   # print("Wikipedia page that has a Table that doesn't hold films: ", film.text)                   
                    t = [s[1].strip(), s[0].strip(), wiki_url]         
                #print(t, "table 2")
                if re.search('\n', t[1]):
                    t = t[1].split('\n')
                    t = [t[1], t[0], wiki_url]
                #testing_list.append(t)
                director_films.add(tuple(t))                     
        #Filmography Section has Number Movie (year) format
        elif film.text[:1].isdigit() and re.search('\(\d\d\d\d\)', film.text):
            film = film.text.rsplit(r')', 1)[0]
            film = film.replace('(', ' ', 1)
            director_films.add((film[:-6].strip(), film[-4:].strip(), wiki_url))
            #print(film, "1")
        #Filmography Section has Movie (year) format
        elif  ')' in film:
            film = film.text.rsplit(r' ', 1)[0]
            film = film.replace('(', ' ', 1)
            director_films.add((film[:-6].strip(), film[-4:].strip(), wiki_url))
            #print(film, "2")
        #Filmography Section has Year Movie format
        elif film.text[:1].isdigit():
            s = film.text.rsplit('(', 1)[0]
            s = s.split(":")
            #print(s[1], s[0])
            try:           
                director_films.add((s[1].strip(), s[0].strip(), wiki_url))
                #print(film.text, "3")            
            except:
                print( "WILL TRY TO FIX LATER OR IGNORE: ", film.text)
        #Filmography Section has Movie, year format
        else:
            bird = film.text.split(' (', 1)
            tweet = re.search('\d\d\d\d', film.text)            
            try:
                tweet = str(tweet.group())
            except:
                tweet = "0000"
            mxm = bird[0].strip(), tweet.strip(), wiki_url
            #print(mxm, "derp")
            director_films.add(tuple(mxm))

print(director_films)

https://en.wikipedia.org/wiki/Abdisalam_Aato
https://en.wikipedia.org/wiki/Fathia_Absie
https://en.wikipedia.org/wiki/Anita_W._Addison
https://en.wikipedia.org/wiki/Omowale_Akintunde
Couldn't find one for  https://en.wikipedia.org/wiki/Queen_Muhammad_Ali
https://en.wikipedia.org/wiki/Khalik_Allah
https://en.wikipedia.org/wiki/Madeline_Anderson
https://en.wikipedia.org/wiki/M._K._Asante
https://en.wikipedia.org/wiki/Philip_Atwell
Couldn't find one for  https://en.wikipedia.org/wiki/Sam_Bailey_(director)
https://en.wikipedia.org/wiki/Mya_Baker
WILL TRY TO FIX LATER OR IGNORE:  1994 - Warrior Queens - Baker's first documentary short which was shown at the Dusable Museum of African American History in Chicago.[6]
WILL TRY TO FIX LATER OR IGNORE:  2010 - Maya Illusion is a short film starring Angelo Boyke and Sophia Loren Coffee; Directed by Mya Baker and Co-Directed by Amy Werber. This film short is an about a young woman who falls in love with a pretty face to find out later that the pret

## This makes a list of Director films from Wikipedia page

## This searches through people and tries to match person to Director via Movies
## It then adds all the movies of a director to a Movie List
## I have to make it so that the personID is added to the names list

In [124]:
dfMovies = pd.DataFrame(columns=['Movie_ID', 'Title', 'Person_ID', 'Name'])


In [100]:
testdf = pd.DataFrame(director_films)
testdf.to_csv('testdf.csv')

In [134]:
ia = Cinemagoer()
person_id = []
found = False
#people = ia.search_person(names)
#people = ia.search_person(wiki_name)

for i in range(300, 311):
    people = ia.search_person(names[i][0])
    found = False
    for person in people:
        if found == False:
            print(person['name'])        
            works = ia.get_person(person.personID)
            #print (works['data']['filmography'])
            try:
                for job in works['filmography'].keys():
                    if job == 'director': 
                        print('Person ID:', person.personID, '\tDirector:', person['name'], '\t# Job:', job, )
                        for movie in works['filmography'][job]:
                            #print (movie, " ", movie['year'])                        
                            if next((i for i, v in enumerate(director_films) if v[0] == movie['title'] and v[1] == str(movie['year'])), None) and found == False:                        #if next((i for i, v in enumerate(director_films) if v[0] == movie['title'] and v[1] == str(movie['year'])), None): # (re.search(my_regex, v[0]) and v[i] == str(movie['year'])))):
                                #print("test 1:", '\tID:%s Title:%s Year: %s' % (movie.movieID, movie['title'], movie['year']))  
                                found = True
                                #dfMovies.loc[len(dfMovies.index)] = [movie.movieID, movie['title'], person.personID]                             
                                for m in works['filmography'][job]:
                                    print(m['title'],'\t', m['year'])
                                    dfMovies.loc[len(dfMovies.index)] = [m.movieID, m['title'], person.personID, person['name']] 
                                #     temp_list = (m.movieID, m['title'], m['year'], person.personID)
                                #     print(temp_list)
                                #     print('\tID:%s Title:%s Year: %s' % (test.movieID, test['title'], test['year']))
                                                                
                            elif next((i for i, v in enumerate(director_films) if v[0] == movie['title']), None) and found == False:
                                #print ("test 2:", '\tID:%s Title:%s Year: %s' % (movie.movieID, movie['title'], movie['year']) )
                                found = True
                                #dfMovies.loc[len(dfMovies.index)] = [movie.movieID, movie['title'], person.personID]   
                                for m in works['filmography'][job]:
                                    print(m['title'],'\t', m['year'])
                                    dfMovies.loc[len(dfMovies.index)] = [m.movieID, m['title'], person.personID, person['name']]   
                                #     temp_list = (m.movieID, m['title'], m['year'], person.personID)
                                #     print(temp_list)
                                #     print('\tID:%s Title:%s Year: %s' % (test.movieID, test['title'], test['year'])) 
                                    
                            else:
                                #break
                                pass
            except:
                print('Person ID:', person.personID, '\tName:    ', person['name'], '\t# Job:', job, "(Doesn't have a filmography)")



Roger Ross Williams
Person ID: 2141317 	Director: Roger Ross Williams 	# Job: director
Fred Williamson
Person ID: 0004365 	Director: Fred Williamson 	# Job: director
Person ID: 0004365 	Name:     Fred Williamson 	# Job: director (Doesn't have a filmography)
Allen Willis
Allison Williams
Allen Williams
Person ID: 0930006 	Director: Allen Williams 	# Job: director
Jen Willis
Diana Allen Williams
Len Willis
Ellen Willis
Ellen Willis
Allen Williamson
Ben Willis
William Allen Young
Person ID: 0950165 	Director: William Allen Young 	# Job: director
Michael Allen Willis
Willie Allen
Glen Willis
Jalen Willis
Helen Willis
Glen Willis
Ellen Willio
J. Allen Williams
Person ID: 2468144 	Director: J. Allen Williams 	# Job: director
Ben Willis
Kevin Willmott
Person ID: 0932551 	Director: Kevin Willmott 	# Job: director
No Place Like Home 	 2022
William Allen White: What's the Matter with Kansas 	 2020
The 24th 	 2020
The Noble Narrative 	 2019
The Profit 	 2017
Fast Break: The Legendary John McLendo

In [135]:
dfMovies

,Movie_ID,Title,Person_ID,Name
0,13814666,Pursued,7280958,Fathia Absie
1,4629690,The Lobby,7280958,Fathia Absie
2,0433539,Copshop,0011697,Anita W. Addison
3,0209069,Judging Amy,0011697,Anita W. Addison
4,0186948,Deep in My Heart,0011697,Anita W. Addison
...,...,...,...,...
592,7608248,A Million Little Things,0937306,Bille Woodruff
593,14507354,The Endgame,0937306,Bille Woodruff
594,11041332,Yellowjackets,0937306,Bille Woodruff
595,3743822,Fear the Walking Dead,0937306,Bille Woodruff


In [136]:
dfMovies.to_csv("Movies.csv")

### This goes through all the people listed in names and prints their ID and name
### will setup to ....

In [ ]:
# test_list = []
# for director in names:
#    people = ia.search_person(director) #searches imdb database for director matches
#    print(type(people))
#    #print('\n\n')
#    #movie = ia.get_movie('0133093')
#    #print('Directors:')
#    for person in people:                # for each person in the database check to see if there is an exact match with 
#        if person['name'] in director:   # director then print their id along with their name
#            print(person.personID, person['name'])
#            test_list.append([person.personID, person['name']])
#            #person_id.append(person.personID)

In [ ]:
a = [(1,"juca"),(22,"james"),(53,"xuxa"),(44,"delicia")]
[x[1] for x in a]

['juca', 'james', 'xuxa', 'delicia']

In [ ]:
re.split('\W+', 'Words, words, words.')

['Words', 'words', 'words', '']